In [ ]:
from google.colab import drive
drive.mount('/content/drive', True)

Mounted at /content/drive


## Prepare Dataset for training

### Install dependencies to use YOLOv8 library

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 28.9 MB/s eta 0:00:00


### Install Roboflow python package and download COCO dataset

<img src="https://cocodataset.org/images/coco-logo.png" />

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="####################") # For security reasons I would prefer not to reveal my api key
project = rf.workspace("jacob-solawetz").project("microsoft-coco")
dataset = project.version(7).download("yolov8")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.114, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to Microsoft-COCO-7 in yolov8:: 100%|██████████| 242836/242836 [00:28<00:00, 8625.62it/s] 


### Edit COCO dataset to be:
- Only one class (person)
- Filter only images containing people

In [ ]:
import os
import shutil
import json
from typing import List

def update_dataset_for_specific_classes(label_ids:List[str], old_path:str, new_path:str):
    # create the new folder
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        os.makedirs(os.path.join(new_path, "images"))
        os.makedirs(os.path.join(new_path, "labels"))

    # go over the images folder
    for filename in os.listdir(os.path.join(old_path, 'images')):
        image_id = filename.rsplit('.', 1)[0]

        # check if the file exists in the labels folder
        file_path = os.path.join(old_path, 'labels', f"{image_id}.txt")
        if not os.path.exists(file_path):
            continue

        # open the corresponding label file
        label_index = 0
        updated = ""
        with open(file_path, "r") as f:
            # read the lines of the file
            lines = f.readlines()
            # go over the lines
            for line in lines:
                # get the label id
                label_id = line.split(' ')[0]
                # check if the label is in the labels we want to keep
                if label_id in label_ids:
                    updated += f"{label_index}  {str(line.split(' ', 1)[1])}"
            
            if updated != "":
                # copy the image to the new folder
                shutil.copy(os.path.join(old_path, 'images', filename), os.path.join(new_path, 'images', filename))

                with open(os.path.join(new_path, 'labels', f"{image_id}.txt"), "w") as new_f:
                    new_f.write(updated)

In [ ]:
# Update train set
train_path = "/content/Microsoft-COCO-7/train"
train_new_path = "/content/Microsoft-COCO-7/train_new"
update_dataset_for_specific_classes(["50"], train_path, train_new_path)

# Update validation set
valid_path = "/content/Microsoft-COCO-7/valid"
valid_new_path = "/content/Microsoft-COCO-7/valid_new"
update_dataset_for_specific_classes(["50"], valid_path, valid_new_path)

# Update test set
test_path = "/content/Microsoft-COCO-7/test"
test_new_path = "/content/Microsoft-COCO-7/test_new"
update_dataset_for_specific_classes(["50"], test_path, test_new_path)

## Custom training YOLOv8n

In [ ]:
# YAML file is edited so that 'person' is the only class
yaml_file_path = "/content/drive/MyDrive/\"Tal's Folder\"/\"COCO dataset 2\"/Microsoft-COCO-7/data.yaml"

In [ ]:
epochs=200
imgsz=640
!yolo detect train data={yaml_file_path} model=yolov8n.yaml epochs={epochs} name=yolov8n_coco_person imgsz={imgsz}


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

### A problem with colab happened during training. Therefore, I will continue from the same point of the network to 150 additional epochs

In [ ]:
epochs=150
imgsz=640
best_model_path ="runs/detect/yolov8n_coco_person/weights/best.pt"
!yolo detect train data={yaml_file_path} model={best_model_path} pretrained=True epochs={epochs} name=yolov8n_coco_person imgsz={imgsz}

New https://pypi.org/project/ultralytics/8.0.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.114 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=runs/detect/yolov8n_coco_person/weights/best.pt, data=/content/drive/MyDrive/Tal's Folder/COCO dataset 2/Microsoft-COCO-7/data.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_coco_person, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

## Test and compare original model and custom training model

### Using other dataset to test and compare several YOLO models
Found in Roboflow - https://universe.roboflow.com/hmahadik/surveillance-1g9be

In [ ]:
project = rf.workspace("hmahadik").project("surveillance-1g9be")
dataset = project.version(1).download("yolov8")

In [ ]:
test_yolo_models_YAML_path = "/content/Surveillance-1/data.yaml"

### YOLOv8n (pretrained)

In [ ]:
!yolo detect val data={test_yolo_models_YAML_path} model=yolov8n.pt

Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients
val: Scanning /content/Surveillance-1/train/labels... 363 images, 0 backgrounds, 0 corrupt: 100% 363/363 [00:00<00:00, 1512.99it/s]
val: New cache created: /content/Surveillance-1/train/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 23/23 [00:05<00:00,  4.20it/s]
                   all        363        702      0.851      0.714      0.823      0.469
                person        363        702      0.851      0.714      0.823      0.469
Speed: 0.5ms preprocess, 3.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val11


### YOLOv8s (pretrained)

In [ ]:
# YOLOv8s (pretrained)
!yolo detect val data={test_yolo_models_YAML_path} model=yolov8s.pt

Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients
val: Scanning /content/Surveillance-1/train/labels.cache... 363 images, 0 backgrounds, 0 corrupt: 100% 363/363 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 23/23 [00:06<00:00,  3.45it/s]
                   all        363        702      0.883      0.796      0.873      0.509
                person        363        702      0.883      0.796      0.873      0.509
Speed: 0.5ms preprocess, 6.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val15


### YOLOv8m (pretrained)

In [ ]:
# YOLOv8m (pretrained)
!yolo detect val data={test_yolo_models_YAML_path} model=yolov8m.pt

Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients
val: Scanning /content/Surveillance-1/train/labels.cache... 363 images, 0 backgrounds, 0 corrupt: 100% 363/363 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 23/23 [00:09<00:00,  2.38it/s]
                   all        363        702      0.913      0.805      0.893      0.531
                person        363        702      0.913      0.805      0.893      0.531
Speed: 0.5ms preprocess, 14.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val16


### YOLOv8n (custom training)

In [ ]:
# YOLOv8n (custom training)
!yolo detect val data={test_yolo_models_YAML_path} model=/content/RescueSign_YOLOv8n_custom.pt

Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /content/Surveillance-1/train/labels.cache... 363 images, 0 backgrounds, 0 corrupt: 100% 363/363 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 23/23 [00:05<00:00,  4.56it/s]
                   all        363        702      0.908      0.786      0.892      0.517
Speed: 0.5ms preprocess, 3.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val17


# **Conclusion**
## **According to the tests on different models, we reached the following results:**
- Our model yields accurate results - Very close to the mAP50 (mean average precision) of the YOLOv8m model even though it is 8.6 times smaller (parameters) and 50 layers less.
- Because it is significantly smaller than the m model it is also almost twice as fast